<a href="https://colab.research.google.com/github/FarrazNouval/fcc-machine-learning-projects/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-10-26 13:21:08--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  97.3MB/s    in 0.3s    

2022-10-26 13:21:09 (97.3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_books.isna().sum()

isbn      0
title     0
author    1
dtype: int64

In [ ]:
df_ratings.isna().sum()

user      0
isbn      0
rating    0
dtype: int64

In [ ]:
df_books = df_books.dropna(axis=0)

In [ ]:
book_count = df_ratings.isbn.value_counts()
book_count = book_count[book_count >= 100].index.to_list()

user_count = df_ratings.user.value_counts()
user_count = user_count[user_count >= 200].index.to_list()

df_ratings = df_ratings[df_ratings.isbn.isin(book_count)]
df_ratings = df_ratings[df_ratings.user.isin(user_count)]

df_ratings


,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0
...,...,...,...
1147304,275970,0804111359,0.0
1147436,275970,140003065X,0.0
1147439,275970,1400031346,0.0
1147440,275970,1400031354,0.0


In [ ]:
test_data = ['The Queen of the Damned (Vampire Chronicles (Paperback))',
             'Catch 22',
             'The Witching Hour (Lives of the Mayfair Witches)', 
             'Interview with the Vampire', 
             'The Tale of the Body Thief (Vampire Chronicles (Paperback))', 
             'The Vampire Lestat (Vampire Chronicles, Book II)', 
             ]


for i in test_data:
  print(df_ratings.isbn.isin(df_books[df_books.title == i].isbn).sum())
  


95
22
88
176
82
124


In [ ]:
df = pd.pivot_table(df_ratings, index='user', columns='isbn', values='rating').fillna(0).T
df

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573227331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1573229326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1573229571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.index = df.join(df_books.set_index('isbn'))['title']
df = df.sort_index()
df.head()


user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.loc['Interview with the Vampire'][:5]

user
254     0.0
2276    0.0
2766    0.0
2977    0.0
3363    0.0
Name: Interview with the Vampire, dtype: float32

In [ ]:
nn = NearestNeighbors(metric='cosine')
nn.fit(df.values)

NearestNeighbors(metric='cosine')

In [ ]:
distances, indices = nn.kneighbors([df.loc['The Queen of the Damned (Vampire Chronicles (Paperback))'].values], 
                                   n_neighbors=6)

distances

array([[0.        , 0.51784116, 0.53763384, 0.73450685, 0.74486566,
        0.7939835 ]], dtype=float32)

In [ ]:
df.iloc[indices[0]].index.values

array(['The Queen of the Damned (Vampire Chronicles (Paperback))',
       'The Vampire Lestat (Vampire Chronicles, Book II)',
       'The Tale of the Body Thief (Vampire Chronicles (Paperback))',
       'Interview with the Vampire',
       'The Witching Hour (Lives of the Mayfair Witches)', 'Catch 22'],
      dtype=object)

In [ ]:
new_df = pd.DataFrame({'title' : df.iloc[indices[0]].index.values,
                       'distance' : distances[0]})

new_df


,title,distance
0,The Queen of the Damned (Vampire Chronicles (P...,0.000000
1,"The Vampire Lestat (Vampire Chronicles, Book II)",0.517841
2,The Tale of the Body Thief (Vampire Chronicles...,0.537634
3,Interview with the Vampire,0.734507
4,The Witching Hour (Lives of the Mayfair Witches),0.744866
5,Catch 22,0.793984


In [ ]:
new_df = new_df.sort_values('distance', ascending=False)

In [ ]:
feature = new_df.title.values[-1]


In [ ]:
target = [[i, j] for i, j in zip(new_df.title.values[:-1], new_df.distance.values[:-1])]

target

[['Catch 22', 0.7939835],
 ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566],
 ['Interview with the Vampire', 0.73450685],
 ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384],
 ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]

In [ ]:
recommendation_format = [feature, target]
recommendation_format

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', 0.7939835],
  ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566],
  ['Interview with the Vampire', 0.73450685],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  dist, idx = nn.kneighbors([df.loc[book]], n_neighbors=6)
  nn_df = pd.DataFrame({'title' : df.iloc[idx[0]].index.values,
                       'distance' : dist[0]}).sort_values('distance', ascending=False)
  
  targets = [[i, j] for i, j in zip(nn_df.title.values, nn_df.distance.values)]
  recommended_books = [book, targets]

  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864], ["Where the Heart Is (Oprah's Book Club (Paperback))", 0.0]]]
You passed the challenge! 🎉🎉🎉🎉🎉
